In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime

**<font color='orange'>eseguire questo notebook da Jupyter Notebook e _non_ da Jupyter Lab</font>**

# Use case reale: `MiMocko` 🛵

Il business vuole provare a capire quali siano le abitudini di viaggio degli utenti.

Carichiamo il dataset dei viaggi.

In [ ]:
path = '../../../data'

In [ ]:
viaggi = pd.read_csv(
    f'{path}/viaggi.csv',
    sep='*',
    decimal=','
)
viaggi.head(2)

## Estraiamo le features di interesse
La nostra analisi mira a capire se esistano dei pattern di utilizzo a seconda delle ore del giorno e del giorno della settimana. In quale colonna è contenuta questa informazione?

In [ ]:
columns = ['idUtente', 'timestampRitiro']
data = viaggi[columns]
data.head()

Dalla colonna `timestampRitiro` è possibile estrarre le feature di interesse, ovvero:
* giorno della settimana
* fascia oraria

Il timestamp di ritiro è in realtà una lista di più valori, iniziamo calcolando il timestamp medio per ogni viaggio.

In [ ]:
data.loc[:, 'timestampRitiro'] = data['timestampRitiro'].apply(lambda x: datetime.fromtimestamp(np.mean(list(map(lambda t: pd.to_datetime(t).timestamp(), x.replace('[', '').replace(']', '').replace("'",'').split(", "))))))

Procediamo quindi con l'estrazione delle feature rilevanti.

In [ ]:
data.loc[:, 'isoWeekDay'] = data['timestampRitiro'].apply(lambda x: x.isoweekday())
data.loc[:, 'hour'] = data['timestampRitiro'].apply(lambda x: x.hour)

In [ ]:
data.head()

Siamo interessati alle abitudini degli utenti, proviamo a valutare per ciascuno di essi, il numero di ritiri per ogni ora/giorno della settimana.

In [ ]:
df = data.groupby(['idUtente', 'isoWeekDay', 'hour'])['timestampRitiro'].count().reset_index()
df

Possiamo ottenere una versione _estesa_ della tabella tramite _pivoting_.

In [ ]:
X = df.pivot_table(index='idUtente', columns=['isoWeekDay', 'hour'], values='timestampRitiro').fillna(0).astype(int)
X.columns = list(map(lambda x: f'Day{x[0]}-H{x[1]}', X.columns))

In [ ]:
X.head()

## Dimensionality reduction
Sarebbe bello poter visualizzare gli utenti in uno scatter plot, ma il dato ottenuto è ad alta dimensionalità e non è quindi possibile.

Al fine di visualizzarlo possiamo procedere alla riduzione di dimensionalità tramite Principal Component Analysis ([PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA)). Questa tecnica ci permette di proiettare dataset ad alta dimensionalità, in _poche_ dimensioni (ad esempio 2).

Per maggiori dettagli su questa tecnica: [link](https://sebastianraschka.com/Articles/2015_pca_in_3_steps.html).

In [ ]:
from IPython.core.display import HTML
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.utils import estimator_html_repr

Prima di procedere alla proiezione dei dati tramite PCA, è buona norma standardizzare i dati. Fortunatamente scikit-learn ci mette a disposizione uno [strumento potente](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) che serve a costruire _sequenze_ di operazioni. Vediamo come realizzare quindi una sequenza di due step:
1. Scaling delle features
2. Riduzione di dimensionalità

In [ ]:
pipe = Pipeline([('scaler', StandardScaler()),
                 ('decomposition', PCA(n_components=3))])

HTML(estimator_html_repr(pipe))

In [ ]:
X_r = pipe.fit_transform(X)

Visualizziamo prima i dati in 2D.

In [ ]:
plt.plot(X_r[:,0], X_r[:,1], 'o')
plt.xlabel(r'$x_0$')
plt.ylabel(r'$x_1$');

Cosa osserviamo?

Tentiamo la stessa visualizzazione anche in 3D, anche se non sempre è d'aiuto.

In [ ]:
%matplotlib widget

fig = plt.figure(dpi=100)
ax = fig.add_subplot(projection='3d')

ax.scatter(X_r[:,0], X_r[:,1], X_r[:, 2]);

Cosa possiamo concludere?

## Clustering
A quanto pare, il nostro dataset è suddiviso in almeno 5 diversi gruppi di utenti, ovvero di _cluster_. La _cluster analysis_ è sicuramente una delle aree più affascinanti nell'ambito dell'unsupervised learning. Un ottimo punto di partenza per iniziare ad approfondire l'argomento è la [guida](https://scikit-learn.org/stable/modules/clustering.html) di scikit-learn.

Il nostro primo approccio, sarà quello di raggruppare gli utenti usando un semplice [K-means](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans). Questa tecnica, basata su centroidi, metodo ricerca un numero fissato di cluster nel dataset, cercando di aggregare punti _vicini_ per distanza Euclidea.

Gli approcci basati su distanza Euclidea tendono a funzionare peggio in presenza di dati ad alta dimensionalità, tale fenomeno è noto come [_curse of dimensionality_](https://en.wikipedia.org/wiki/Curse_of_dimensionality). Per cercare di _mitigare_ tale effetto, possiamo nuovamente scegliere di usare PCA (con un bassi numero arbitrario di componenti) come step di preprocessing all'algorimo di clustering vero e proprio.

Maggiori dettagli su questo metodo: [qui](https://towardsdatascience.com/k-means-clustering-explained-4528df86a120).

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
pipe = Pipeline([('scaler', StandardScaler()),
                 ('decomposition', PCA(n_components=15)),
                 ('clustering', KMeans(n_clusters=5))])

HTML(estimator_html_repr(pipe))

In [ ]:
y = pipe.fit_predict(X)

Andiamo a ripetere la visualizzazione precedente, colorando ogni punto con il valore stimato in `y` dal nostro algorimo di clustering.

In [ ]:
%matplotlib widget

fig = plt.figure(dpi=100)
ax = fig.add_subplot(projection='3d')

ax.scatter(X_r[:,0], X_r[:,1], X_r[:, 2], c=y);

# Interpretazione cluster
Interpretare l'esito del clustering è, in generale, complicato e può portare a trarre conclusioni errate. In questo caso, simulato, proviamo a capire se è possibile ricondurre l'etichetta assegnata dal clustering ad un pattern di comportamento.

Iniziamo con l'assegnare l'etichetta ad ogni utente.

In [ ]:
dfy = (pd.Series(dict(zip(X.index, y)))
         .reset_index()
         .rename({0: 'cluster', 'index': 'idUtente'}, axis=1))
clustered_df = pd.merge(df, dfy, left_on=['idUtente'], right_on=['idUtente'])
clustered_df.head()

Al fine di semplificare la nostra interpretazione, andiamo a creare due nuove features, una che distingue le ore notturne da quelle diurne ed una'altra che distingua i giorni lavorativi da quelli infrasettimanali.

In [ ]:
clustered_df['week'] = (clustered_df['isoWeekDay'] >= 6).map({True: 'weekend', False: 'workingDay'})
clustered_df['shift'] = ((clustered_df['hour'] >= 19) | (clustered_df['hour'] <= 6)).map({True: 'night', False: 'day'})
clustered_df.head()

Proviamo quindi a raggruppare secondo etichetta assegnata ed abitudini di ritiro del mezzo.

In [ ]:
clustered_df.groupby(['cluster', 'shift', 'week'])['timestampRitiro'].count().to_frame()

Cosa possiamo concludere?